# Keyword Search

In [1]:
!pip install cohere
!pip install weaviate-client

  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ------------ --------------------------- 0.8/2.5 MB 4.2 MB/s eta 0:00:01
   ------------------------- -------------- 1.6/2.5 MB 4.0 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 3.8 MB/s eta 0:00:00
Using cached fsspec-2025.5.1-py3-none-any.whl (199 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.3 MB 4.2 MB/s eta 0:00:01
   ---------------- ----------------------- 1.8/4.3 MB 4.8 MB/s eta 0:00:01
   -------------------------- ------------- 2.9/4.3 MB 4.9 MB/s eta 0:00:01
   -------------------------------------- - 4.2/4.3 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.2 MB 4.8 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.2 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
   --------- ------------------------------ 0.8/3.4 MB 5.6 MB/s eta 0:00:01
   --------------------- ------------------ 1.8/3.4 MB 5.6 MB/s eta 0:00:01
   ------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 6.31.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

Importing Weaviate to access the Wikipedia database.

In [2]:
import weaviate
auth_config = weaviate.auth.AuthApiKey(
    api_key=os.environ['WEAVIATE_API_KEY'])

In [4]:
from weaviate.classes.init import Auth

weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())


True


# Keyword Search

In [5]:
!pip show weaviate-client

Name: weaviate-client
Version: 4.15.4
Summary: A python native Weaviate client
Home-page: https://github.com/weaviate/weaviate-python-client
Author: Weaviate
Author-email: hello@weaviate.io,
License: BSD 3-clause
Location: d:\anaconda\envs\tf\lib\site-packages
Requires: authlib, deprecation, grpcio, grpcio-health-checking, grpcio-tools, httpx, pydantic, validators
Required-by: 


d:\Anaconda\envs\tf\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
d:\Anaconda\envs\tf\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
d:\Anaconda\envs\tf\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


In [7]:
from weaviate.classes.config import Configure, Property, DataType

client.collections.create(
    name="Articles",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="url", data_type=DataType.TEXT),
        Property(name="text", data_type=DataType.TEXT),
        Property(name="lang", data_type=DataType.TEXT)
    ],
    vectorizer_config=Configure.Vectorizer.none()
)

UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name Articles already exists'}]}.

In [28]:
articles = client.collections.get("Articles")

articles.data.insert({
    "title": "World Cup Final",
    "url": "https://en.wikipedia.org/wiki/2022_FIFA_World_Cup_final",
    "text": "The FIFA World Cup final is the most viewed televised event globally.",
    "lang": "en"
})

articles.data.insert({
    "title": "Olympic Opening Ceremony",
    "url": "https://en.wikipedia.org/wiki/2024_Summer_Olympics_opening_ceremony",
    "text": "The Olympic Games opening ceremony attracts billions of viewers worldwide.",
    "lang": "en"
})

UUID('3509f111-cf15-4d8c-8d01-773654bd6e9f')

In [19]:
from weaviate.classes.query import Filter

def keyword_search(query, results_lang="en"):
    filters = Filter.by_property("lang").equal(results_lang)

    response = (
        client.collections.get("Articles")
        .query.bm25(
            query=query,
            filters=filters,
            limit=5,
            return_properties=["title", "url", "text"]
        )
    )
    return response.objects

results = keyword_search("What is the most viewed televised event?")
for obj in results:
    print(obj.properties)


{'title': 'World Cup Final', 'url': 'https://example.com/world-cup', 'text': 'The FIFA World Cup final is the most viewed televised event globally.'}
{'title': 'World Cup Final', 'url': 'https://en.wikipedia.org/wiki/2022_FIFA_World_Cup_final', 'text': 'The FIFA World Cup final is the most viewed televised event globally.'}
{'title': 'World Cup Final', 'url': 'https://en.wikipedia.org/wiki/2022_FIFA_World_Cup_final', 'text': 'The FIFA World Cup final is the most viewed televised event globally.'}


In [35]:
print(client.is_ready())

True


In [22]:
client.collections.list_all()

{'Articles': _CollectionConfigSimple(name='Articles', description=None, generative_config=None, properties=[_Property(name='title', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none', vectorizer_configs=None), _Property(name='url', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none', vectorizer_configs=None), _Property(name='text', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none', vectorizer_configs=None), _Property(name='lang', description=None,

### Modifying the search options
- Other languages: `en, de, fr, es, it, ja, ar, zh, ko, hi`

In [23]:
properties = ["text", "title", "url", 
             "views", "lang"]

In [24]:
def print_result(result):
    """ Print results with colorful formatting """
    for i, item in enumerate(result):
        print(f'item {i}')
        for key in item.properties.keys():
            print(f"{key}: {item.properties.get(key)}")
            print()
        print()


In [25]:
print_result(results)

In [26]:
results = keyword_search("What is the most viewed televised event?", results_lang="de")

In [27]:
print(results)

[]


In [28]:
from weaviate.classes.data import DataObject

client.collections.get("Articles").data.insert_many([
    {
        "title": "Super Bowl is the most viewed televised event",
        "url": "https://example.com/super-bowl",
        "text": "The Super Bowl attracts millions of viewers worldwide.",
        "lang": "en"
    },
    {
        "title": "Olympics Opening Ceremony",
        "url": "https://example.com/olympics",
        "text": "The Olympics opening ceremony is one of the biggest TV events.",
        "lang": "en"
    },
    {
        "title": "Fußball-Weltmeisterschaft Finale",
        "url": "https://example.com/wm-finale",
        "text": "Das Finale der Fußball-Weltmeisterschaft zieht Millionen Zuschauer an.",
        "lang": "de"
    }
])

BatchObjectReturn(_all_responses=[UUID('41c1072b-ef00-4079-ada1-a1c3344c6f50'), UUID('79092532-b67e-4fa2-b92e-241ea1eb9506'), UUID('b4c86e0f-a9fe-46ae-8067-c0bc812ecffa')], elapsed_seconds=0.07684874534606934, errors={}, uuids={0: UUID('41c1072b-ef00-4079-ada1-a1c3344c6f50'), 1: UUID('79092532-b67e-4fa2-b92e-241ea1eb9506'), 2: UUID('b4c86e0f-a9fe-46ae-8067-c0bc812ecffa')}, has_errors=False)

In [29]:
results = keyword_search("What is the most viewed televised event?", results_lang="de")
print(results)

[]


In [32]:
response = collection.query.bm25(
    query="the",
    limit=5,
    return_properties=["title", "text", "url", "lang"]
)

for obj in response.objects:
    print(obj.properties)

In [33]:
client.collections.create(
    name="NewsItems",
    properties=[
        Property(name="headline", data_type=DataType.TEXT),
        Property(name="url", data_type=DataType.TEXT),
        Property(name="body", data_type=DataType.TEXT),
        Property(name="lang", data_type=DataType.TEXT)
    ],
    vectorizer_config=Configure.Vectorizer.none()
)

In [34]:
collection = client.collections.get("NewsItems")
collection.data.insert_many([
    {
        "headline": "The Super Bowl",
        "url": "https://example.com/superbowl",
        "body": "The Super Bowl is the most watched TV event in the US.",
        "lang": "en"
    },
    {
        "headline": "Olympic Games",
        "url": "https://example.com/olympics",
        "body": "The Olympic Games attract billions of viewers worldwide.",
        "lang": "en"
    },
    {
        "headline": "Weltmeisterschaft",
        "url": "https://example.com/wm",
        "body": "Die Fußball-Weltmeisterschaft ist eines der größten TV-Ereignisse.",
        "lang": "de"
    }
])

BatchObjectReturn(_all_responses=[UUID('a050678c-1a8d-4a45-bfc2-81eb264b18e5'), UUID('f720f146-a306-42c6-b750-c52942fa30af'), UUID('69c79d74-6ab2-426d-8f70-ac6e2a199296')], elapsed_seconds=0.6188235282897949, errors={}, uuids={0: UUID('a050678c-1a8d-4a45-bfc2-81eb264b18e5'), 1: UUID('f720f146-a306-42c6-b750-c52942fa30af'), 2: UUID('69c79d74-6ab2-426d-8f70-ac6e2a199296')}, has_errors=False)

In [36]:
from weaviate.classes.query import Filter

def keyword_search(query, results_lang="en"):
    filters = Filter.by_property("lang").equal(results_lang)

    response = (
        client.collections.get("NewsItems")
        .query.bm25(
            query=query,
            filters=filters,
            limit=5,
            return_properties=["headline", "url", "body"]
        )
    )
    return response.objects

results = keyword_search("What is the most viewed televised event?")
for obj in results:
    print(obj.properties)

{'body': 'The Super Bowl is the most watched TV event in the US.', 'url': 'https://example.com/superbowl', 'headline': 'The Super Bowl'}


In [37]:
properties = ["headline", "url", "body", "lang"]

In [38]:
def print_result(result):
    """ Print results with colorful formatting """
    for i, item in enumerate(result):
        print(f'item {i}')
        for key in item.properties.keys():
            print(f"{key}: {item.properties.get(key)}")
            print()
        print()

In [40]:
results = keyword_search("Was ist das meistgesehene Fernsehereignis?", "de")
for obj in results:
    print(obj.properties)

{'body': 'Die Fußball-Weltmeisterschaft ist eines der größten TV-Ereignisse.', 'url': 'https://example.com/wm', 'headline': 'Weltmeisterschaft'}
